In [ ]:
import urllib.parse
import requests
import pandas as pd
import datetime
import io
import re
import progressbar

In [ ]:
startYear = 1996
today = datetime.date.today()
currentYear = today.year
all_dfs = []

In [ ]:
# def buddhistYear(year):
#     return year+543

In [ ]:
# dlType = "interestCSV" # interestCSV or interestXLS
# dateFrom = "01/1/2565" # dd/mm/YYYY * Year in Buddhist calendar. Eg. 13/1/2565
# dateTo = "31/12/2565" # dd/mm/YYYY * Year in Buddhist calendar. Eg. 13/1/2565
# bankValue = "002" # bankId
# bankName = "กรุงเทพ" # bankName
# rdoTypeStr = "1" # 0 for deposit or 1 for loan

def getBOT_URL(dateFrom,dateTo,bankValue,bankName,rdoTypeStr):
    payload = {
        "dlType" : dlType,
        "dateFrom" : dateFrom,
        "dateTo" : dateTo,
        "bankValue" : bankValue,
        "bankName" : bankName,
        "rdoTypeStr" : rdoTypeStr
    }
    # url = "https://www.bot.or.th/thai/statistics/_layouts/application/interest_rate/DownloadFile.aspx?"    
    url = "https://www.bot.or.th/english/statistics/_layouts/application/interest_rate/DownloadFile.aspx?"
    return url+urllib.parse.urlencode(payload)

In [ ]:
bank_df = pd.read_csv("bank_en.csv",sep=";",header=0,names=["bankValue","bankName"],dtype={"bankValue":str,"bankName":str})

In [ ]:
for index,row in bank_df.iterrows():
    bankValue = row.bankValue
    bankName = row.bankName
    bankBar = progressbar.ProgressBar()
    print("Bank : {}".format(bankName))
    for year in bankBar(range(startYear,currentYear+1)):
        dateFrom = "01/01/{}".format(year)
        dateTo = "31/12/{}".format(year)

        if(year==startYear):
            dateFrom="31/01/{}".format(year)
        elif(year==currentYear):
            date = today.day
            month = today.month
            dateTo = "{}/{}/{}".format(date,month,year)

        dlType = "interestCSV"
        rdoTypeStr = "1" 

        url = getBOT_URL(dateFrom,dateTo,bankValue,bankName,rdoTypeStr)
        res = requests.get(url)
        if(re.search("The selected date has no data to generate file.",res.text)):
            print("Skip {} for year {}".format(bankName,year))
        else:
            df = pd.read_csv(io.StringIO(res.text),sep="|",encoding="iso8859_11",skiprows=[0,1])
            df["bank"] = bankName
            all_dfs.append(df)

In [ ]:
consolidated_df = pd.concat(all_dfs)

In [ ]:
consolidated_df.to_csv("./output/loan-interest-rates_EN.csv",index=False)